# TP4: Introduction aux réseaux de neurones
Pour ce TP, nous allons utiliser la librairie PyTorch.

---
Le code qui suit est une fonction qui ne sert qu'à dessiner les réseaux de neurones. **Vous n'avez pas besoin d'en prendre connaissance**. Il faut run la cellule puis passer directement à la partie **Tensors**

In [ ]:
# Heavily based on https://github.com/Prodicode/ann-visualizer

def ann_viz(model, view=True, filename="network.gv"):
    """Vizualizez a Sequential model.

    # Arguments
        model: A Keras model instance.

        view: whether to display the model after generation.

        filename: where to save the vizualization. (a .gv file)

        title: A title for the graph
    """
    from graphviz import Digraph

    input_layer = 0
    hidden_layers_nr = 0
    layer_types = []
    hidden_layers = []
    output_layer = 0
    layers = [layer for layer in model.modules() if type(layer) == torch.nn.Linear]

    for layer in layers:
        if layer == layers[0]:
            input_layer = layer.in_features
            hidden_layers_nr += 1
            if type(layer) == torch.nn.Linear:                
                hidden_layers.append(layer.out_features)
                layer_types.append("Dense")
            else:
                raise Exception("Input error")

        else:
            if layer == layers[-1]:
                output_layer = layer.out_features
            else:
                hidden_layers_nr += 1
                if type(layer) == torch.nn.Linear:

                    hidden_layers.append(layer.out_features)
                    layer_types.append("Dense")
                else:
                    raise Exception("Hidden error")
        last_layer_nodes = input_layer
        nodes_up = input_layer

    g = Digraph("g", filename=filename)
    n = 0
    g.graph_attr.update(splines="false", nodesep="0.5", ranksep="0", rankdir='LR')
    # Input Layer
    with g.subgraph(name="cluster_input") as c:
        if type(layers[0]) == torch.nn.Linear:
            the_label = "Input Layer"
            if layers[0].in_features > 10:
                the_label += " (+" + str(layers[0].in_features - 10) + ")"
                input_layer = 10
            c.attr(color="white")
            for i in range(0, input_layer):
                n += 1
                c.node(str(n))
                c.attr(labeljust="1")
                c.attr(label=the_label, labelloc="bottom")
                c.attr(rank="same")                
                c.node_attr.update(
                    width="0.65",
                    style="filled",                    
                    shape="circle",
                    color=HAPPY_COLORS_PALETTE[3],
                    fontcolor=HAPPY_COLORS_PALETTE[3],
                )
    for i in range(0, hidden_layers_nr):
        with g.subgraph(name="cluster_" + str(i + 1)) as c:
            if layer_types[i] == "Dense":
                c.attr(color="white")
                c.attr(rank="same")
                the_label = f'Hidden Layer {i + 1}'
                if layers[i].out_features > 10:
                    the_label += " (+" + str(layers[i].out_features - 10) + ")"
                    hidden_layers[i] = 10
                c.attr(labeljust="right", labelloc="b", label=the_label)
                for j in range(0, hidden_layers[i]):
                    n += 1
                    c.node(
                        str(n),
                        width="0.65",
                        shape="circle",
                        style="filled",
                        color=HAPPY_COLORS_PALETTE[0],
                        fontcolor=HAPPY_COLORS_PALETTE[0],
                    )
                    for h in range(nodes_up - last_layer_nodes + 1, nodes_up + 1):
                        g.edge(str(h), str(n))
                last_layer_nodes = hidden_layers[i]
                nodes_up += hidden_layers[i]
            else:
                raise Exception("Hidden layer type not supported")

    with g.subgraph(name="cluster_output") as c:
        if type(layers[-1]) == torch.nn.Linear:
            c.attr(color="white")
            c.attr(rank="same")
            c.attr(labeljust="1")
            for i in range(1, output_layer + 1):
                n += 1
                c.node(
                    str(n),
                    width="0.65",
                    shape="circle",
                    style="filled",
                    color=HAPPY_COLORS_PALETTE[4],
                    fontcolor=HAPPY_COLORS_PALETTE[4],
                    
                )
                for h in range(nodes_up - last_layer_nodes + 1, nodes_up + 1):
                    g.edge(str(h), str(n))
            c.attr(label="Output Layer", labelloc="bottom")
            c.node_attr.update(
                color="#2ecc71", style="filled", fontcolor="#2ecc71", shape="circle"
            )

    g.attr(arrowShape="none")
    g.edge_attr.update(arrowhead="none", color="#707070", penwidth="2")
    if view is True:
        g.view()

    return g

---
## 1. Tensors

1. Importez torch et numpy :

In [ ]:
# Import libraries


2. Transformez *data* en tensor :

In [ ]:
# Create some data
data = [[1, 2],[3, 4]]

# From list to tensor


3. Transformez *data* en array puis de array en tensor

In [ ]:
# From list to array

# From array to tensor


4. Créez un tensor avec des valeurs aléatoires (**torch.rand()**) ayant une forme (3,4). Puis affichez ses valeurs, son type (**.dtype**) et le device (**.device**) sur lequel est enregistré ce dernier.

In [ ]:
# Create random tensor of shape (3, 4)


5. Créez 2 tensors:
[[10, 20], [30, 40]]
et [[2, 2], [1, 5]].  
Faites en la somme puis la multiplication.

In [ ]:
# Create 2 tensor

# Elementwise sum and mult


6. Transformez le résultat de la multiplication en array

In [ ]:
# From tensor to array


---
## 2. Build your own MLP


In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt

from torch import nn, optim
import torch.nn.functional as F


%matplotlib inline
%config InlineBackend.figure_format='retina'

# Display graph parameters
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

# Random seed parameters
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

### Préparation des données
Pour tout projet de data science, la première partie, et souvent la plus fastidieuse, est la préparation des données. Une fois cette étape validée, vous pourrez utiliser ces dernières dans un réseau de neurones.

Téléchargez les données au lien suivant: [https://www.kaggle.com/jsphyg/weather-dataset-rattle-package](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package)  
Créez un dossier Data/ et importer les données dans ce dernier.  
Dézippez le fichier dans le dossier Data:

In [ ]:
# Unzip file containing data
# !unzip  Data/archive.zip -d Data/

7. Créez le dataframe *df* à partir des données *weatherAUS.csv*

In [ ]:
# Read data


8. Affichez le nombre de lignes puis de colonnes du dataframe

In [ ]:
# Number of rows

# Number of columns


9. Sélectionnez uniquement les colonnes de *df* qui nous intéresse:  
['Rainfall', 'Humidity3pm', 'Pressure9am', 'RainToday', 'RainTomorrow']

In [ ]:
# Select only relevant columns


10. Checkez les valeurs manquantes puis supprimez les lignes contenant des NaN.

In [ ]:
# Print NaN

# Drop NaN


Dans cette exemple, nous avons assez d'observations à étudier, on va se contenter de supprimer toutes les lignes où des NaN sont présents.
**Note:** cette manière de pratiquer n'est pas très académique mais le but du TP est autre.

11. Les modèles de machine learning n'acceptent que des données numériques. Il faut donc transformer les valeurs qualitatives en quantitatives des colonnes RainToday et RainTomorrow:  
- Yes = 1
- No = 0

S'assurer que les colonnes soient bien de type **int**.

In [ ]:
# Transform string into binary int values


Ensuite, on va découper notre jeu de données en 2 datasets:  
- 1 pour l'entraînement du réseau. Que l'on découpera ensuite en 2 autres datasets (train/validation)
- 1 pour tester notre modèle (test). Ce dernier va nous permettre de calculer les performances de notre modèle sur des données nouvelles, pas utilisée lors de l'entraînement.

12. Utilisez **train_test_split** pour créer deux datasets à partir de *df*: *df_train* et *df_test*

In [ ]:
# Split df into train/validation set (80%/20%) with random_state=RANDOM_SEED


Un des problèmes classique en machine learning est le déséquilibre de classe. Ce problème est souvent à l'origine de piètres performances des modèles. On va regarder ce qu'il en est pour notre jeu de données:

13. Tracez un barplot qui représente la distribution des valeurs de la colonne *df_train.RainTomorrow*. Vous pouvez facilement y parvenir grâce à *sns.countplot()*

In [ ]:
# Countplot


14. Déterminer le pourcentage de 0 et de 1 dans cette même colonnne *df_train.RainTomorrow*

In [ ]:
# In percent, we get:


Notre dataset est énormément déséquilibré! (78%/22%).  
 On va appliquer la technique dites d'oversampling (simple). C'est à dire augmenter la quantité de lignes ayant pour valeur cible 1 pour en avoir que de 1. Ici, on ajoute simplement à df_train des lignes où RainTomorrow == 1.

In [ ]:
# Oversampling
df_train = pd.concat([df_train, df_train[df_train.RainTomorrow == 1], df_train[df_train.RainTomorrow == 1]], axis=0)

15. Tracez l'histogramme qui montre la nouvelle distribution de RainTomorrow et calculez le pourcentage de 0 et de 1.

In [ ]:
# Print again the new balance between rain and no rain


In [ ]:
# In percent, we get:


Maintenant que nos données sont à peu près équilibrées (53%/46%), il faut découper notre jeu de données en jeu de train et validation.

16. Découpez en deux dataset le dataframe df_train (80% de données en train et 20% en validation). Vous pouvez à nouveau utiliser la fonction **train_test_split**, mais cette fois avec df_train et les colonnes pertinentes. df_train[X] pour les variables et df_train[y] pour la valeur cible.

In [ ]:
# Split df_train in train/test set (80%/20%) with random_state=RANDOM_SEED


17. Convertissez X_train, X_test, y_train, y_test en array (*.to_numpy*) puis de array en tensor (*torch.from_numpy*) sous la forme de float (*.float*). Attention les *y_* doivent être *.squeeze*.

In [ ]:
# Convert data from numpy to float tensor


### Multi Layer Perceptron

#### Create the network

Dans cette partie, vous allez utiliser un MLP pour réaliser les prédictions de pluie à l'aide de la variable RainTomorrow. Vous pouvez observer dans la classe MLP les différentes couches de neurones, leurs dimensions et les fonctions d'activations utilisées.

In [ ]:
# Définir la classe MLP
class MLP(nn.Module):

  def __init__(self, n_features):
    super(MLP, self).__init__()
    self.fc1 = nn.Linear(n_features, 5)
    self.fc2 = nn.Linear(5, 3)
    self.fc3 = nn.Linear(3, 1)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    return torch.sigmoid(self.fc3(x))

In [ ]:
# Initialize the network
model = MLP(X_train.shape[1])

In [ ]:
# Print our network
ann_viz(model, view=False)

On peut aussi afficher la structure du modèle:

In [ ]:
# Check model structure
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

18. Maintenant que le réseau est créé, vous allez définir ses hyperparamètres.   
On veut utiliser la fonction de perte **nn.BCELoss** et l'optimiseur **optim.SGD** avec un learning rate de 0.001

In [ ]:
## Hyperparameters

# Choose the loss function

# Choose the optimizer


Ensuite on définit quelques fonctions et listes...

In [ ]:
# Define a wee function to compute accuracy
def calculate_accuracy(y_true, y_pred):
  predicted = y_pred.ge(.5).view(-1)
  return (y_true == predicted).sum().float() / len(y_true)

# Define a function to round numbers
def round_tensor(t, decimal_places=3):
  return round(t.item(), decimal_places)

In [ ]:
# Create some empty lists to pick up some values during training
list_train_loss = []
list_val_loss = []
list_train_acc = []
list_val_acc = []

#### Training

19. C'est ici que l'entraînement du modèle va avoir lieu. Les données seront lues 1000 fois (1000 epochs). Vous allez devoir compléter les lignes de codes et permettre l'entraînement du modèle.

In [ ]:
############## TRAINING ##############
for epoch in range(1000):
    
    # Do prediction on trainset
    y_pred = # Here
    y_pred = torch.squeeze(y_pred)

    # Compute loss
    train_loss = # Here
    
    # For some epoch compute val loss and accuracy
    if epoch % 100 == 0:
      # Accuracy between train and true
      train_acc = # Here
      
      # Prediction on X_validation
      y_val_pred = # Here
      y_val_pred = torch.squeeze(y_val_pred)

      # Compute loss and accuracy on validation
      val_loss = # Here
      val_acc = # Here

      # Save train/val loss/accuracy
      list_train_loss.append(round_tensor(train_loss))
      list_val_loss.append(round_tensor(val_loss))
      list_train_acc.append(round_tensor(train_acc))
      list_val_acc.append(round_tensor(val_acc))

      # Print some informations
      print(
f'''epoch {epoch}
Train set - loss: {round_tensor(train_loss)}, accuracy: {round_tensor(train_acc)}
Validation  set - loss: {round_tensor(val_loss)}, accuracy: {round_tensor(val_acc)}
''')
    
    # All gradients to zero (avoid exploding gradient)
    # Here

    # Backproapagation
    # Here
    
    # Update weights
    # Here

20. Afin d'éviter l'overfitting ou l'underfitting de notre modèle, vous allez analyser les valeurs de perte que relevées pour le trainset et validation set. Utilisez les variables *list_train_loss* et *list__loss* et tracez leur variation en fonction de leur epoch associée. **plt.plot()**

In [ ]:
# Check overfitting (Train Loss vs Test Loss)


#### Test
Le test est une étape d'inférence. Le modèle maintenant entraîné va être capable de faire des prédictions sur des données qu'il n'a jamais vu. Nous pourrons alors déterminer son efficacité.

21. Vous devez donc créer 2 variables issues de *df_test*: *X_test* et *y_test*. Comme précédemment, convertir d'abord *df_test[X]* en array puis en tensor et en float. Pareil pour *df_test[y]*.  
Puis faire les prédiction à l'aide du modèle sur les données *X_test*.

In [ ]:
# Convert data from numpy to tensor


22. Utiliser la fonction **classification_report** pour obtenir toutes les métriques qui vous aideront à qualifier les performances de votre modèle.

In [ ]:
# Do prediction on testset


#### Confusion Matrix

23. Ici, je vous ai créé la matrice de confusion associée aux résultats du test. Conclure sur l'efficacité de votre modèle:

In [ ]:
cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, index=classes, columns=classes)

hmap = sns.heatmap(df_cm, annot=True, fmt="d")
hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
plt.ylabel('True label')
plt.xlabel('Predicted label');

In [ ]:
print(f'Number of "No rain" values: {np.count_nonzero(y_test.numpy() == 0)}')
print(f'Number of "Raining" values: {np.count_nonzero(y_test.numpy() == 1)}')

**Note:** avant de passer à la suite du TP, vous pouvez essayer de modifier l'architecture du MLP (nombre de layers, fonction d'activations, ...) que vous avez utilisé pour faire la prédiction et relancer le code en entier. Vous pourrez alors observer des variations de performances.

## 3. A vous de jouer !

Télécharger l'archive au [lien](https://www.kaggle.com/iabhishekofficial/mobile-price-classification/download) suivant et placez là dans un dossier Data2 que vous créerez.  
Après avoir dézippé l'archive, vous trouverez un jeu de données de train et un de test. Vous trouverez une description des données [ici](https://www.kaggle.com/iabhishekofficial/mobile-price-classification?select=train.csv).

24. Sur le même principe que pour la partie **Build your own MLP**, vous devez créer un réseau de neurones multi-couche afin de prédire la classe de prix des téléphones du jeu de test. Vous êtes libre dans la préparation des données, le choix des variables et des paramètres (optimizer, learning rate, fonction d'activation, ...).  
Utilisez la fonction [**classification_report()**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) pour afficher les résultats de votre prédiction.